In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
words = open('data/raw/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

X, Y = [], []
w = words[0]
print(w)
context = [0] * block_size
for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
print(X[0], '-->', Y[0])
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
[0, 0, 0] --> 5


In [5]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])

In [6]:
C = torch.rand((27,2))

In [7]:
C[5] # indexing is faster than matrix mult

tensor([0.7087, 0.7297])

In [8]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([0.7087, 0.7297])

In [9]:
X.shape

torch.Size([5, 3])

In [10]:
C[X].shape

torch.Size([5, 3, 2])

In [11]:
X[3,2].item()

13

In [12]:
C[13]

tensor([0.9775, 0.5601])

In [13]:
C[X][3,2] # emb for example 3, 2nd char of context

tensor([0.9775, 0.5601])

In [14]:
emb = C[X]
emb.shape
# dim1 = num of examples (contexts)
# dim2 = contexts
# dim3 = embed

torch.Size([5, 3, 2])

In [15]:
emb # big block is example #x, each row i is emb for char i of the example

tensor([[[0.8707, 0.9683],
         [0.8707, 0.9683],
         [0.8707, 0.9683]],

        [[0.8707, 0.9683],
         [0.8707, 0.9683],
         [0.7087, 0.7297]],

        [[0.8707, 0.9683],
         [0.7087, 0.7297],
         [0.9775, 0.5601]],

        [[0.7087, 0.7297],
         [0.9775, 0.5601],
         [0.9775, 0.5601]],

        [[0.9775, 0.5601],
         [0.9775, 0.5601],
         [0.8902, 0.7857]]])

In [16]:
W1 = torch.randn(6,100) # 100 neurons 6 inputs (the whole example above)
b1 = torch.rand(100)

In [17]:
# wanna do this emb @ W1 + b1
# 5x3x2 6x100 
# 5x6 6x100 -> 5x100

In [18]:
emb[:,0,:].shape # all examples, first letter of context, all embeding 
# first input in paper graph
# we pass 3 inputs: first letter, second, third of context for # X examples

torch.Size([5, 2])

In [19]:
emb[:,0,:]

tensor([[0.8707, 0.9683],
        [0.8707, 0.9683],
        [0.8707, 0.9683],
        [0.7087, 0.7297],
        [0.9775, 0.5601]])

In [20]:
print(torch.cat((emb[:,0,:], emb[:,1,:], emb[:,2,:]), dim=1).shape)
torch.cat((emb[:,0,:], emb[:,1,:], emb[:,2,:]), dim=1) # glue columns
# first 5x2 is block above (first letters)
# second block is second letter...

torch.Size([5, 6])


tensor([[0.8707, 0.9683, 0.8707, 0.9683, 0.8707, 0.9683],
        [0.8707, 0.9683, 0.8707, 0.9683, 0.7087, 0.7297],
        [0.8707, 0.9683, 0.7087, 0.7297, 0.9775, 0.5601],
        [0.7087, 0.7297, 0.9775, 0.5601, 0.9775, 0.5601],
        [0.9775, 0.5601, 0.9775, 0.5601, 0.8902, 0.7857]])

In [21]:
# not hardcoded
torch.cat(torch.unbind(emb, 1), 1)

tensor([[0.8707, 0.9683, 0.8707, 0.9683, 0.8707, 0.9683],
        [0.8707, 0.9683, 0.8707, 0.9683, 0.7087, 0.7297],
        [0.8707, 0.9683, 0.7087, 0.7297, 0.9775, 0.5601],
        [0.7087, 0.7297, 0.9775, 0.5601, 0.9775, 0.5601],
        [0.9775, 0.5601, 0.9775, 0.5601, 0.8902, 0.7857]])

In [22]:
a = torch.arange(20)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])

In [23]:
a.view(5,4) # this is fast

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])

In [24]:
type(a.view(5,4)) # a regular tensor

torch.Tensor

In [25]:
emb.shape

torch.Size([5, 3, 2])

In [26]:
emb.view(5,6)

tensor([[0.8707, 0.9683, 0.8707, 0.9683, 0.8707, 0.9683],
        [0.8707, 0.9683, 0.8707, 0.9683, 0.7087, 0.7297],
        [0.8707, 0.9683, 0.7087, 0.7297, 0.9775, 0.5601],
        [0.7087, 0.7297, 0.9775, 0.5601, 0.9775, 0.5601],
        [0.9775, 0.5601, 0.9775, 0.5601, 0.8902, 0.7857]])

In [27]:
# emb.view(emb.shape[0], 6)
emb.view(-1, 6) # given that (-1)x6 = 5x3x2 can complete (-1)

tensor([[0.8707, 0.9683, 0.8707, 0.9683, 0.8707, 0.9683],
        [0.8707, 0.9683, 0.8707, 0.9683, 0.7087, 0.7297],
        [0.8707, 0.9683, 0.7087, 0.7297, 0.9775, 0.5601],
        [0.7087, 0.7297, 0.9775, 0.5601, 0.9775, 0.5601],
        [0.9775, 0.5601, 0.9775, 0.5601, 0.8902, 0.7857]])

In [28]:
h = emb.view(-1, 6) @ W1 + b1

In [29]:
b1.shape, (emb.view(-1, 6) @ W1).shape
# 5 x 100
# 0->1->5 x 100

(torch.Size([100]), torch.Size([5, 100]))

In [30]:
h.shape
# h[0] all activations for first example

torch.Size([5, 100])

In [31]:
W2 = torch.randn(100, 27) # 100 neurons 6 inputs (the whole example above)
b2 = torch.rand(27)

In [32]:
logits = h @ W2 + b2 
logits.shape

torch.Size([5, 27])

In [33]:
counts = logits.exp()
counts.sum(dim=1, keepdim=True).shape

torch.Size([5, 1])

In [34]:
prob = counts / counts.sum(dim=1, keepdim=True)
prob.shape

torch.Size([5, 27])

In [35]:
sum(prob[0])

tensor(1.)

In [36]:
Y

tensor([ 5, 13, 13,  1,  0])

In [37]:
prob[torch.arange(5), Y] # should be ~= 1 when trained

tensor([4.3830e-03, 4.8756e-21, 1.2270e-20, 1.7581e-10, 8.3317e-19])

In [38]:
loss = -prob[torch.arange(5), Y].log().mean()
loss

tensor(32.4276)

# Summary of the neural net

In [39]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

X, Y = [], []
for w in words[:5]:
  context = [0] * block_size
  for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [40]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [41]:
sum(p.nelement() for p in parameters)

3481

In [42]:
# forward pass
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(dim=1, keepdim=True)
loss = - prob[torch.arange(X.shape[0]), Y].log().mean()
loss

tensor(17.7697)

In [43]:
X.shape[0]

32